In [ ]:
# Instalar bibliotecas (caso necessário no Google Colab)
!pip install pandas matplotlib seaborn pyarrow

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
# 1. Carregar o ficheiro Parquet (aqui na área de ficheiros)
def carregar_dados(caminho_ficheiro):
    try:
        dados = pd.read_parquet(caminho_ficheiro, engine="pyarrow")
        print("Dados carregados com sucesso!")
        return dados
    except Exception as e:
        print(f"Erro ao carregar os dados: {e}")
        return None

# Substitua o caminho abaixo pelo caminho do seu ficheiro Parquet
dados = carregar_dados("dados_sensores_5000.parquet")

In [ ]:
# 2. Análise exploratória inicial
if dados is not None:
    # Mostrar as primeiras linhas dos dados
    print("\nPrimeiras linhas do conjunto de dados:")
    print(dados.head())

    # Mostrar informações gerais sobre os dados
    print("\nInformações gerais dos dados:")
    print(dados.info())

    # Mostrar estatísticas descritivas
    print("\nEstatísticas descritivas:")
    print(dados.describe())

    # Verificar se há valores ausentes
    print("\nValores ausentes por coluna:")
    print(dados.isnull().sum())


In [ ]:
# 3. Tratamento de dados (se necessário caso haja valores em falta)
if dados is not None:
    # Substituir valores ausentes por zero ou pelo valor médio, conforme necessário
    dados.fillna({
        "energia_kwh": dados["energia_kwh"].mean(),
        "agua_m3": dados["agua_m3"].mean(),
        "co2_emissoes": dados["co2_emissoes"].mean()
    }, inplace=True)

    print("\nDados após tratamento de valores ausentes:")
    print(dados.isnull().sum())

In [ ]:
# 4. Análises específicas deste ponto em diante:
# Energia, Água e CO2 por setor
setor_analise = dados.groupby("setor").agg({
    "energia_kwh": "sum",
    "agua_m3": "sum",
    "co2_emissoes": "sum"
}).sort_values("energia_kwh", ascending=False)

print("\nConsumo por setor:")
print(setor_analise)

In [ ]:
# 5. Identificar empresas com consumo excessivo de água
# Utilizou-se um multiplicador de 1.75 pois está a meio do sugerido (entre 1,5 e 2)
consumo_excessivo = dados[dados["agua_m3"] > dados["agua_m3"].mean() + 1.75 * dados["agua_m3"].std()]
print("\nEmpresas com consumo excessivo de água:")
print(consumo_excessivo)

In [ ]:
# 6. Gráficos para análise visual
sns.set(style="whitegrid")

# Gráfico: Consumo de energia por setor
plt.figure(figsize=(10, 6))
setor_analise["energia_kwh"].plot(kind="bar", color="skyblue")
plt.title("Consumo de Energia por Setor")
plt.ylabel("Energia (kWh)")
plt.xlabel("Setor")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 6.1 Gráfico: Consumo de água por setor
plt.figure(figsize=(10, 6))
setor_analise["agua_m3"].plot(kind="bar", color="green")
plt.title("Consumo de Água por Setor")
plt.ylabel("Água (m³)")
plt.xlabel("Setor")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 6.2 Gráfico: Emissões de CO2 por setor
plt.figure(figsize=(10, 6))
setor_analise["co2_emissoes"].plot(kind="bar", color="red")
plt.title("Emissões de CO2 por Setor")
plt.ylabel("CO2 (emissões)")
plt.xlabel("Setor")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 6.3 Comparação entre setores
sns.pairplot(dados, hue="setor", diag_kind="kde", corner=True)
plt.suptitle("Comparação entre Setores")
plt.show()

In [ ]:
# 6.4 Boxplot para identificar empresas com consumo fora do padrão
plt.figure(figsize=(12, 6))
sns.boxplot(data=dados, x="setor", y="agua_m3", hue="setor", palette="coolwarm", legend=False)
plt.title("Distribuição de Consumo de Água por Setor")
plt.ylabel("Consumo de Água (m³)")
plt.xlabel("Setor")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Calcular a matriz de correlação
correlation_matrix = dados[['energia_kwh', 'agua_m3', 'co2_emissoes']].corr()

# Visualizar a matriz de correlação usando um mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de correlação entre energia, água, e emissões de CO2')
plt.show()

# Representação numérica da matriz de correlação
print(correlation_matrix)

In [ ]:
# Calcular a média e desvio padrão por setor
sector_stats = dados.groupby('setor').agg({
    'energia_kwh': ['mean', 'std'],
    'agua_m3': ['mean', 'std'],
    'co2_emissoes': ['mean', 'std']
})

# Função para identificar outliers com base em z-score
def find_outliers(group, column, sector, threshold=1.5):  # Added 'sector' as argument
    z_scores = (group[column] - sector_stats.loc[sector, (column, 'mean')]) / sector_stats.loc[sector, (column, 'std')]
    return group[abs(z_scores) > threshold]

# Identificar outliers para cada recurso e setor
outliers_list = []
for sector, group in dados.groupby('setor'):
    outliers_list.append(find_outliers(group, 'energia_kwh', sector))  # Pass 'sector' to the function
    outliers_list.append(find_outliers(group, 'agua_m3', sector))       # Pass 'sector' to the function
    outliers_list.append(find_outliers(group, 'co2_emissoes', sector))  # Pass 'sector' to the function

outliers = pd.concat(outliers_list)

# Exibir as empresas outliers
print(outliers)

In [ ]:
# Agrupar empresas que tem perfis semelhantes de consumo
# Selecionar as colunas para clustering
features = dados[['energia_kwh', 'agua_m3', 'co2_emissoes']]

# Padronizar os dados
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Criar um modelo K-Means com 3 clusters (ou ajustar número de clusters conforme desejado)
kmeans = KMeans(n_clusters=3, random_state=42)  # random_state para reprodutibilidade

# Ajustar o modelo aos dados
kmeans.fit(scaled_features)

# Obter os rótulos dos clusters para cada empresa
dados['cluster'] = kmeans.labels_

# Visualizar os clusters (gráfico de dispersão)
plt.figure(figsize=(10, 6))
sns.scatterplot(data=dados, x='energia_kwh', y='agua_m3', hue='cluster', palette='viridis')
plt.title('Agrupamento de Empresas por Consumo de Recursos')
plt.xlabel('Consumo de Energia (kWh)')
plt.ylabel('Consumo de Água (m³)')
plt.show()

# Analisar as características de cada cluster (média de consumo por cluster)
cluster_means = dados.groupby('cluster')[['energia_kwh', 'agua_m3', 'co2_emissoes']].mean()
print(cluster_means)

In [ ]:
# Calcular estatísticas descritivas por setor
sector_stats = dados.groupby('setor')[['energia_kwh', 'agua_m3', 'co2_emissoes']].describe()
print(sector_stats)

# Visualizar o consumo médio por setor com gráficos de barras
sector_means = dados.groupby('setor')[['energia_kwh', 'agua_m3', 'co2_emissoes']].mean()

sector_means.plot(kind='bar', figsize=(12, 6))
plt.title('Consumo Médio de Recursos por Setor')
plt.ylabel('Consumo Médio')
plt.xticks(rotation=45)
plt.show()

# Identificar outliers em cada setor (usando boxplots)
plt.figure(figsize=(12, 6))
sns.boxplot(data=dados, x='setor', y='energia_kwh')  # Substituir 'energia_kwh' por outras colunas para análise
plt.title('Distribuição do Consumo de Energia por Setor')
plt.ylabel('Consumo de Energia (kWh)')
plt.xticks(rotation=45)
plt.show()

# Comparar a distribuição do consumo entre setores (usando histogramas ou KDE plots)
sns.displot(data=dados, x='energia_kwh', hue='setor', kind='kde', fill=True)  # Substituir 'energia_kwh' por outras colunas
plt.title('Distribuição do Consumo de Energia por Setor')
plt.xlabel('Consumo de Energia (kWh)')
plt.show()